In [12]:
import pandas as pd
import random 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm

import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene
from scipy.stats import kstest
from scipy.stats import skew

import math 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA


import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (20,10) 
pd.options.display.max_columns = None

##### <font color=green> En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

In [13]:
df= pd.read_pickle("datos/cars_price_norm_est.pkl")
df.head(2) 

,Levy,Manufacturer,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Color,Airbags,Price_log
0,1.766963,LEXUS,-0.411248,Jeep,Yes,Hybrid,3.5,0.237134,1.375313,Automatic,4x4,04,Silver,1.329406,9.497622
1,0.943936,CHEVROLET,-0.173337,Jeep,No,Petrol,3,0.273438,1.375313,Tiptronic,4x4,04,Black,0.319668,9.718422


In [14]:
df.dtypes

Levy                 float64
Manufacturer          object
Prod. year           float64
Category              object
Leather interior      object
Fuel type             object
Engine volume         object
Mileage              float64
Cylinders            float64
Gear box type         object
Drive wheels          object
Doors               category
Color                 object
Airbags              float64
Price_log            float64
dtype: object

##### <font color=green> Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

##### <font color=green> 📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

Hemos detectado que para hacer el anova necesitamos que no haya espacios entre las palabras de los nombres de las columnas, al llegar a este punto, nos damos cuenta que es algo que ya tendriamos que haber modficado al hacer la primera limpieza... :( perooo, más vale tarde que nunca :) 

In [15]:
df.columns 

Index(['Levy', 'Manufacturer', 'Prod. year', 'Category', 'Leather interior',
       'Fuel type', 'Engine volume', 'Mileage', 'Cylinders', 'Gear box type',
       'Drive wheels', 'Doors', 'Color', 'Airbags', 'Price_log'],
      dtype='object')

In [16]:
df.rename(columns={'Prod. year':'Prod_year', 'Leather interior':'Leather_interior', 'Fuel type':'Fuel_type', 'Engine volume':'Engine_volume', 'Gear box type':'Gear_box_type', 'Drive wheels':'Drive_wheels'}, inplace=True)

In [17]:
df.describe()

,Levy,Prod_year,Mileage,Cylinders,Airbags,Price_log
count,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000,9877.000000
mean,0.007800,0.007608,0.001800,0.002635,0.000912,9.557538
std,1.002296,1.000678,1.004885,1.000446,1.000478,0.994712
min,-1.255121,-2.552451,-0.889269,-3.090707,-1.699806,6.308098
25%,-1.255121,-0.411248,-0.477477,-0.411095,-0.690069,9.182147
50%,0.127391,0.064574,-0.168633,-0.411095,-0.185200,9.699411
75%,0.645832,0.778309,0.164355,-0.411095,1.329406,10.130185
max,3.391414,1.967865,5.287479,10.307355,2.339143,12.640792


In [18]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de los coches
lm = ols('Price_log ~ Levy + Manufacturer + Prod_year + Category + Leather_interior + Fuel_type + Engine_volume + Mileage + Cylinders + Gear_box_type + Drive_wheels + Doors + Color + Airbags', data = df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
Manufacturer,56.0,857.767234,15.317272,21.037473,1.129883e-196
Category,10.0,367.702975,36.770298,50.502083,1.775789e-99
Leather_interior,1.0,53.452635,53.452635,73.414403,1.211620e-17
Fuel_type,6.0,152.132306,25.355384,34.824296,6.882716e-42
Engine_volume,97.0,770.108997,7.939268,10.904170,1.368125e-151
Gear_box_type,3.0,447.092582,149.030861,204.686102,1.002071e-128
Drive_wheels,2.0,11.229123,5.614561,7.711307,4.504920e-04
Doors,2.0,6.518478,3.259239,4.476395,1.139790e-02
Color,15.0,56.607691,3.773846,5.183180,2.004102e-10
Levy,1.0,0.155885,0.155885,0.214100,6.435829e-01


In [19]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Price_log   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     19.09
Date:                Thu, 06 Oct 2022   Prob (F-statistic):               0.00
Time:                        00:04:24   Log-Likelihood:                -12348.
No. Observations:                9877   AIC:                         2.509e+04
Df Residuals:                    9680   BIC:                         2.651e+04
Df Model:                         196                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         9.3281      0.566     16.493      0.000       8.219      10.437
Manufacturer[T.ALFA ROMEO]       -0.2513      0.528     -0.476      0.634      -1.287       0.784
Manufacturer[T.ASTON MARTIN]      0.6635      0.494      1.344      0.179      -0.304       1.631
Manufacturer[T.AUDI]             -0.0348      0.318     -0.109      0.913      -0.659       0.589
Manufacturer[T.BENTLEY]           0.6149      1.092      0.563      0.573      -1.525       2.755
Manufacturer[T.BMW]               0.0281      0.313      0.090      0.928      -0.585       0.641
Manufacturer[T.BUICK]            -0.1822      0.432     -0.422      0.673      -1.029       0.664
Manufacturer[T.CADILLAC]          0.0469      0.426      0.110      0.912      -0.787       0.881
Manufacturer[T.CHEVROLET]         0.2858      0.310      0.922      0.356      -0.322       0.893
Manufacturer[T.CHRYSLER]         -0.0090      0.367     -0.024      0.980      -0.729       0.711
Manufacturer[T.CITROEN]          -0.1044      0.434     -0.241      0.810      -0.955       0.747
Manufacturer[T.DAEWOO]           -0.0307      0.335     -0.091      0.927      -0.688       0.627
Manufacturer[T.DAIHATSU]         -0.7099      0.473     -1.501      0.133      -1.637       0.217
Manufacturer[T.DODGE]            -0.5240      0.330     -1.587      0.113      -1.171       0.123
Manufacturer[T.FERRARI]           2.3782      0.798      2.980      0.003       0.814       3.942
Manufacturer[T.FIAT]              0.2842      0.338      0.840      0.401      -0.379       0.947
Manufacturer[T.FORD]              0.2304      0.310      0.744      0.457      -0.377       0.837
Manufacturer[T.GAZ]              -0.6443      0.911     -0.708      0.479      -2.429       1.141
Manufacturer[T.GMC]              -0.1144      0.423     -0.271      0.787      -0.943       0.714
Manufacturer[T.HAVAL]            -0.0303      0.920     -0.033      0.974      -1.833       1.773
Manufacturer[T.HONDA]             0.6295      0.311      2.026      0.043       0.020       1.239
Manufacturer[T.HUMMER]            0.6997      0.619      1.130      0.259      -0.514       1.914
Manufacturer[T.HYUNDAI]           0.7238      0.308      2.347      0.019       0.119       1.328
Manufacturer[T.INFINITI]          0.5101      0.379      1.346      0.178      -0.233       1.253
Manufacturer[T.JAGUAR]            0.5460      0.357      1.528      0.127      -0.154       1.246
Manufacturer[T.JEEP]              0.8234      0.319      2.580      0.010       0.198       1.449
Manufacturer[T.KIA]               0.4803      0.312      1.540      0.124      -0.131       1.092
Manufacturer[T.LANCIA]            0.1508      0.913      0.165      0.869      -1.639       1.940
Manufacturer[T.LAND ROVER]        0.5927     

#### <font color=violet> Análisis de Summary

Luego de analizar el p-valor para cada una de las variables catgóricas, observamos que hay algunas que no afectan nuestra variable `Precio`, es decir, hemos sacado el promedio del p-valor de estas y nos ha dado mayor a 0.05 (`p-valor>0.05 aceptamos nuestra hipótesis nula`). Por esta razón hemos decidido eliminar las variables `Manufacturer`, `Engine_Volumen`, `Color`, `Doors` para continuar con nuestro proceso de encoding.

In [20]:
df.drop(["Manufacturer", "Engine_volume", "Color", "Doors"], axis = 1, inplace = True)
df.head(2)

,Levy,Prod_year,Category,Leather_interior,Fuel_type,Mileage,Cylinders,Gear_box_type,Drive_wheels,Airbags,Price_log
0,1.766963,-0.411248,Jeep,Yes,Hybrid,0.237134,1.375313,Automatic,4x4,1.329406,9.497622
1,0.943936,-0.173337,Jeep,No,Petrol,0.273438,1.375313,Tiptronic,4x4,0.319668,9.718422


In [21]:
df.to_pickle('datos/cars_price_norm_est_an.pkl')

In [22]:
df.to_csv('datos/cars_price_norm_est_an.csv')
